# Assignment 1: ADH (2013) with composition adjustment

## Code Preliminaries

In [2]:
!pip install econtools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.1 MB/s eta 0:00:0000:01


In [10]:
from pathlib import Path
import pandas as pd
from linearmodels.iv import IV2SLS, compare
from io import StringIO
import warnings
import numpy as np
from econtools import group_id

In [11]:
# Functions for weighted aggregation

def WtSum(df:pd.core.frame.DataFrame, cols:list, weight_col:str, by_cols:list, 
          outw=False, mask=None):
    '''Weighted sum'''
    
    out = df[[*cols, weight_col, *by_cols]].copy()
    out[[*cols, weight_col]] = out[[*cols, weight_col]].astype(np.float64) #for sum precision
    
    if mask is not None:
        out = out[mask]

    for c in cols:
        out[c] = out[c] * out[weight_col]
    
    if outw:
        return out.groupby(by_cols)[[*cols, weight_col]].sum()
    else:
        return out.groupby(by_cols)[cols].sum()

def WtMean(df:pd.core.frame.DataFrame, cols:list, weight_col:str, by_cols:list, 
           mask=None):
    '''Weighted mean'''    
    
    out_list = []
    for c in cols:
        out = df[[c, weight_col, *by_cols]].copy()
        out[[c, weight_col]] = out[[c, weight_col]].astype(np.float64) #for sum precision

        if mask is not None:
            out = out[mask]
            
        out = out[~np.isnan(out[c])] #remove missings
        out.loc[:,c] = out.loc[:,c] * out.loc[:,weight_col] #multiply by weights
        out = out.groupby(by_cols)[[c, weight_col]].sum() #sum
        out.loc[:,c] = out.loc[:,c] / out.loc[:,weight_col] # divide by total weights

        out_list.append(out[c])

    return pd.concat(out_list, axis=1)

## Prepare the Composition-Adjusted Data

In [12]:
mainp = Path('/Users/augusto/Dropbox/UCLA Classes/Teaching/econ424_S22/assig1')

In [13]:
# Column definitions:
pd.read_stata('usa_00137.dta', iterator=True).variable_labels()

{'year': 'census year',
 'statefip': 'state (fips code)',
 'puma': 'public use microdata area',
 'gq': 'group quarters status',
 'perwt': 'person weight',
 'sex': 'sex',
 'age': 'age',
 'race': 'race [general version]',
 'bpl': 'birthplace [general version]',
 'educ': 'educational attainment [general version]',
 'empstat': 'employment status [general version]',
 'ind1990': 'industry, 1990 basis',
 'wkswork1': 'weeks worked last year',
 'wkswork2': 'weeks worked last year, intervalled',
 'uhrswork': 'usual hours worked per week',
 'incwage': 'wage and salary income'}

In [14]:
df = pd.read_stata('usa_00137.dta', convert_categoricals=False)
# Keep those aged 16-64 and not in group quarters:
df = df[(df.age>=16) & (df.age<=64) & (df.gq<=2)].copy()

In [15]:
df.head()

,year,statefip,puma,gq,perwt,sex,age,race,bpl,educ,empstat,ind1990,wkswork1,wkswork2,uhrswork,incwage
0,1990,2,101,1,12,1,37,1,23,6,1,360,31.0,3,50,15500
1,1990,2,101,1,15,2,28,1,2,4,3,151,46.0,4,40,7975
3,1990,2,101,1,8,1,44,1,6,7,1,880,52.0,6,20,18000
4,1990,2,101,1,7,2,41,7,16,7,3,641,0.0,0,0,0
5,1990,2,101,1,15,2,20,1,22,6,1,820,20.0,2,65,2500


#### Define (128) groups over which we CA: gender (2) x US born (2) x age bin (4) x education bin (4) x race bin (2)

In [16]:
df['agebin'] = pd.cut(df.age, bins=[15,27,39,51,64], labels=False)
df['educbin'] = pd.cut(df.educ, bins=[-1,5,6,9,11], labels=False)
df['college'] = np.where((df.educ>9) & (df.educ<=11), 1, 0)
df['white'] = np.where(df.race==1, 1, 0)
df['native'] = np.where(df.bpl<=99, 1, 0)
df['male'] = np.where(df.sex==1, 1, 0)

In [17]:
df.drop(columns=['age','educ','race','bpl','sex'], inplace=True)
group_cols = ['male', 'native', 'agebin', 'educbin', 'white']
df = group_id(df, cols=group_cols, merge=True, name='groups')

In [18]:
df.head()

,year,statefip,puma,gq,perwt,empstat,ind1990,wkswork1,wkswork2,uhrswork,incwage,agebin,educbin,college,white,native,male,groups
0,1990,2,101,1,12,1,360,31.0,3,50,15500,1,1,0,1,1,1,0
1,1990,2,101,1,15,3,151,46.0,4,40,7975,1,0,0,1,1,0,1
3,1990,2,101,1,8,1,880,52.0,6,20,18000,2,2,0,1,1,1,2
4,1990,2,101,1,7,3,641,0.0,0,0,0,2,2,0,0,1,0,3
5,1990,2,101,1,15,1,820,20.0,2,65,2500,0,1,0,1,1,0,4


#### Get geography to cz level

In [19]:
df.loc[(df.statefip==22)&(df.puma==77777), 'puma'] = 1801 #Katrina data issue
df['PUMA'] = df['statefip'].astype(str).str.zfill(2) + df['puma'].astype(str).str.zfill(4)
df['PUMA'] = df['PUMA'].astype('int')

In [21]:
df1990 = df[df.year==1990].merge(pd.read_stata('cw_puma1990_czone.dta'),
                                 left_on='PUMA', right_on='puma1990')
df2000 = df[df.year!=1990].merge(pd.read_stata('cw_puma2000_czone.dta'),
                                 left_on='PUMA', right_on='puma2000')
df = pd.concat([df1990, df2000])
df['perwt'] = df['perwt'] * df['afactor']
del df1990; del df2000

#### Aggregate to cz x group x year level

In [22]:
# Employment status:
df['emp'] = np.where(df.empstat==1, 1, 0)
df['unemp'] = np.where(df.empstat==2, 1, 0)
df['nilf'] = np.where(df.empstat==3, 1, 0)
# Manufacturing employment:
df['manuf'] = np.where((df.emp==1) & (df.ind1990>=100) & (df.ind1990<400), 1, 0)
df['nonmanuf'] = np.where((df.emp==1) & ((df.ind1990<100) | (df.ind1990>=400)), 1, 0)
# Filling in weeks worked for 2008 ACS (using midpoint):
df.loc[df.wkswork2==1, 'wkswork1'] = 7
df.loc[df.wkswork2==2, 'wkswork1'] = 20
df.loc[df.wkswork2==3, 'wkswork1'] = 33
df.loc[df.wkswork2==4, 'wkswork1'] = 43.5
df.loc[df.wkswork2==5, 'wkswork1'] = 48.5
df.loc[df.wkswork2==6, 'wkswork1'] = 51
# Log weekly wage:
df['lnwkwage'] = np.log(df.incwage/df.wkswork1)
df.loc[df['lnwkwage']==-np.inf, 'lnwkwage'] = np.nan
# Hours:
df['hours'] = df['uhrswork'] * df['wkswork1']

df.drop(columns=['empstat','wkswork2','incwage'], inplace=True)

/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [23]:
wmean_cols = ['lnwkwage']                                    #columns to take weighted mean
sum_cols = ['manuf','nonmanuf','emp','unemp','nilf','hours'] #columns to sum

In [24]:
by_cols=['czone','year','groups',*group_cols,'college']
df_cgy = pd.concat(
    [WtMean(df, cols=wmean_cols, weight_col='perwt', by_cols=by_cols),
     WtSum(df, cols=sum_cols, weight_col='perwt', by_cols=by_cols, outw=True)]
    , axis=1
)
df_cgy.rename(columns={'perwt':'pop'}, inplace=True)

for c in ['manuf','nonmanuf','unemp','nilf']:
    df_cgy['{}_share'.format(c)] = df_cgy[c] / df_cgy['pop']

for c in [*sum_cols,'pop']:
    df_cgy['ln{}'.format(c)] = np.log(df_cgy[c])
    df_cgy.loc[df_cgy['ln{}'.format(c)]==-np.inf, 'ln{}'.format(c)] = np.nan
    
del df
df_cgy = df_cgy.reset_index().set_index(['czone','year','groups'])

/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/a1/anaconda3/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarn

In [25]:
df_cgy.head()

male  native  agebin  educbin  white  college  lnwkwage  \
czone year groups                                                            
100.0 1990 0          1       1       1        1      1        0  5.905023   
           1          0       1       1        0      1        0  5.238786   
           2          1       1       2        2      1        0  6.228145   
           3          0       1       2        2      0        0  5.434239   
           4          0       1       0        1      1        0  5.007850   

                         manuf     nonmanuf           emp  ...  \
czone year groups                                          ...   
100.0 1990 0       6160.805960  9598.628952  15759.434913  ...   
           1       1719.524990  1861.706991   3581.231980  ...   
           2       3156.005985  4572.614977   7728.620963  ...   
           3          0.000000   106.067999    106.067999  ...   
           4       2402.573983  6684.686969   9087.260952  ...   

                   nonmanuf_share  unemp_share  nilf_share   lnmanuf  \
czone year groups                                                      
100.0 1990 0             0.546923     0.059587    0.042452  8.725963   
           1             0.231380     0.049567    0.505344  7.449803   
           2             0.543082     0.026300    0.055785  8.057063   
           3             0.685340     0.251992    0.062668       NaN   
           4             0.455925     0.083023    0.297185  7.784296   

                   lnnonmanuf     lnemp   lnunemp    lnnilf    lnhours  \
czone year groups                                                        
100.0 1990 0         9.169376  9.665195  6.952500  6.613435  17.346662   
           1         7.529249  8.183462  5.988503  8.310427  15.692632   
           2         8.427841  8.952686  5.400147  6.152077  16.676266   
           3         4.664080  4.664080  3.663562  2.272023  12.367806   
           4         8.807575  9.114629  7.104364  8.379601  16.592259   

                      lnpop  
czone year groups            
100.0 1990 0       9.772823  
           1       8.992942  
           2       9.038336  
           3       5.041921  
           4       9.593001  

[5 rows x 25 columns]

#### Aggregate to cz x year level

We now have a database at the level of the commuting zone ($i$) by year ($t$) by group ($g$). For the regressions we need data at the level of commuting zone by year ($it$). We will construct composition-adjusted measures as

$$L_{it}^{CA} = \sum_g \bar{\theta}_{ig} L_{igt}$$

where the time-invariant weights $\bar{\theta}_{ig}$ are the average across periods of hours weights:

$$
\bar{\theta}_{ig} = \frac{1}{3} \left( \theta_{ig1990}+ \theta_{ig2000}+ \theta_{ig2008}\right)
$$
where
$$
\theta_{igt} = hours_{igt} \Big/ \left( \sum_g hours_{igt} \right).
$$

    
Note that $\sum_g \bar{\theta}_{ig}=1$. 

In [26]:
# Create weights
df_w = df_cgy.reset_index()[['czone','year','groups','hours']].copy()

# Deal with missing obs as zeros (which they are):
df_w = df_w.set_index(['czone','year','groups']).unstack(level=[1,2], fill_value=0.0).stack(level=[1,2])

df_w['weight_cgt'] = df_w['hours'] / df_w.groupby(['czone','year'])['hours'].transform('sum')
df_w['weight_cg'] = df_w.groupby(['czone','groups'])['weight_cgt'].transform('mean')

df_cgy = pd.concat([df_cgy, 
                    df_w[['weight_cg']].rename(columns={'weight_cg':'weight'})
                   ], axis=1)

del df_w

/var/folders/_5/mn8jxggn2fg23grkrzzwk0940000gn/T/ipykernel_9609/3212948915.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_w = df_w.set_index(['czone','year','groups']).unstack(level=[1,2], fill_value=0.0).stack(level=[1,2])


In [27]:
# Create the average log wages across various aggregations within a czone x year
def fun(m): 
    return WtMean(df_cgy.reset_index(), cols=['lnwkwage'], 
                  weight_col='weight', by_cols=['czone','year'], mask=m)
col_mask = df_cgy.reset_index().college==1
ncol_mask = df_cgy.reset_index().college==0
male_mask = df_cgy.reset_index().male==1
female_mask = df_cgy.reset_index().male==0

df_cy = pd.concat(
    [fun(None),
     fun(col_mask).rename(columns={'lnwkwage':'lnwkwage_col'}),
     fun(ncol_mask).rename(columns={'lnwkwage':'lnwkwage_ncol'}),
     fun(male_mask).rename(columns={'lnwkwage':'lnwkwage_male'}),
     fun(female_mask).rename(columns={'lnwkwage':'lnwkwage_female'}),
     fun(col_mask & male_mask).rename(columns={'lnwkwage':'lnwkwage_col_male'}),
     fun(col_mask & female_mask).rename(columns={'lnwkwage':'lnwkwage_col_female'}),
     fun(ncol_mask & male_mask).rename(columns={'lnwkwage':'lnwkwage_ncol_male'}),
     fun(ncol_mask & female_mask).rename(columns={'lnwkwage':'lnwkwage_ncol_female'})
    ], axis=1
)

In [28]:
# Create CA shares
share_cols = ['manuf_share', 'nonmanuf_share', 'unemp_share','nilf_share']
def fun(m): 
    return WtMean(df_cgy.reset_index(), cols=share_cols, 
                  weight_col='weight', by_cols=['czone','year'], mask=m)
col_mask = df_cgy.reset_index().college==1
ncol_mask = df_cgy.reset_index().college==0

df_cy = pd.concat(
    [df_cy,
     fun(None),
     fun(col_mask).add_suffix('_col'),
     fun(ncol_mask).add_suffix('_ncol'),
    ], axis=1
)

In [29]:
# Create CA log counts
# (We are taking a weighted average of logs. One could alternatively take the log of weighted averages)
count_cols = ['lnmanuf','lnnonmanuf','lnemp','lnunemp','lnnilf','lnpop']
df_cy = pd.concat([df_cy,
                   WtMean(df_cgy.reset_index(), cols=count_cols, 
                          weight_col='weight', by_cols=['czone','year'])
                  ], axis=1)

In [30]:
df_cy.head()

lnwkwage  lnwkwage_col  lnwkwage_ncol  lnwkwage_male  \
czone year                                                         
100.0 1990  5.744054      6.195654       5.635246       5.956934   
      2000  6.079296      6.549042       5.966177       6.254768   
      2008  6.220569      6.750330       6.092771       6.391578   
200.0 1990  5.720078      6.254544       5.631585       5.935252   
      2000  6.058228      6.552655       5.977792       6.211522   

            lnwkwage_female  lnwkwage_col_male  lnwkwage_col_female  \
czone year                                                            
100.0 1990         5.445086           6.420949             5.890197   
      2000         5.832479           6.752622             6.273352   
      2008         5.980048           6.946428             6.485444   
200.0 1990         5.420252           6.469751             5.971634   
      2000         5.842394           6.722158             6.327919   

            lnwkwage_ncol_male  lnwkwage_ncol_female  manuf_share  ...  \
czone year                                                         ...   
100.0 1990            5.847165              5.335083     0.232911  ...   
      2000            6.137225              5.723366     0.188186  ...   
      2008            6.260515              5.854522     0.155468  ...   
200.0 1990            5.849265              5.325315     0.201472  ...   
      2000            6.130861              5.760145     0.190017  ...   

            manuf_share_ncol  nonmanuf_share_ncol  unemp_share_ncol  \
czone year                                                            
100.0 1990          0.250266             0.476374          0.047103   
      2000          0.200300             0.493775          0.036102   
      2008          0.161416             0.507933          0.054505   
200.0 1990          0.213847             0.495845          0.053644   
      2000          0.206094             0.514652          0.041271   

            nilf_share_ncol   lnmanuf  lnnonmanuf     lnemp   lnunemp  \
czone year                                                              
100.0 1990         0.226257  7.544271    8.399336  8.769013  5.801741   
      2000         0.269822  7.469443    8.586439  8.893505  5.751241   
      2008         0.276147  7.269781    8.628068  8.878520  6.088994   
200.0 1990         0.236664  6.369200    7.368492  7.670625  4.964864   
      2000         0.237983  6.534029    7.686570  7.979600  5.079836   

              lnnilf     lnpop  
czone year                      
100.0 1990  7.189200  9.073486  
      2000  7.628538  9.243600  
      2008  7.677698  9.261179  
200.0 1990  6.272496  7.992839  
      2000  6.596962  8.296156  

[5 rows x 27 columns]

#### Create 10-year equivalent changes

In [31]:
cols = df_cy.columns.to_list()

# Reshape to wide format:
df_cy = df_cy.reset_index().pivot_table(index='czone', columns='year')

# Compute decadal differences:
for c in cols:
    df_cy['D{}'.format(c),1990] = df_cy[c,2000] - df_cy[c,1990]
    df_cy['D{}'.format(c),2000] = (df_cy[c,2008] - df_cy[c,2000])*(10/7)  
    
# Reshape back to long format:
df_cy = df_cy.stack().drop(columns=cols)

/var/folders/_5/mn8jxggn2fg23grkrzzwk0940000gn/T/ipykernel_9609/1177492673.py:12: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_cy = df_cy.stack().drop(columns=cols)


In [32]:
df_cy.head()

Dlnwkwage  Dlnwkwage_col  Dlnwkwage_ncol  Dlnwkwage_male  \
czone year                                                             
100.0 1990   0.335242       0.353387        0.330931        0.297834   
      2000   0.201818       0.287554        0.180848        0.195443   
      2008        NaN            NaN             NaN             NaN   
200.0 1990   0.338151       0.298112        0.346207        0.276270   
      2000   0.147773       0.185134        0.140884        0.138770   

            Dlnwkwage_female  Dlnwkwage_col_male  Dlnwkwage_col_female  \
czone year                                                               
100.0 1990          0.387393            0.331672              0.383155   
      2000          0.210813            0.276866              0.302988   
      2008               NaN                 NaN                   NaN   
200.0 1990          0.422142            0.252407              0.356284   
      2000          0.161150            0.212888              0.153171   

            Dlnwkwage_ncol_male  Dlnwkwage_ncol_female  Dmanuf_share  ...  \
czone year                                                            ...   
100.0 1990             0.290060               0.388283     -0.044725  ...   
      2000             0.176130               0.187366     -0.046740  ...   
      2008                  NaN                    NaN           NaN  ...   
200.0 1990             0.281596               0.434830     -0.011456  ...   
      2000             0.126935               0.160845     -0.084099  ...   

            Dmanuf_share_ncol  Dnonmanuf_share_ncol  Dunemp_share_ncol  \
czone year                                                               
100.0 1990          -0.049966              0.017402          -0.011001   
      2000          -0.055550              0.020225           0.026289   
      2008                NaN                   NaN                NaN   
200.0 1990          -0.007753              0.018807          -0.012373   
      2000          -0.099917              0.064586           0.013738   

            Dnilf_share_ncol  Dlnmanuf  Dlnnonmanuf    Dlnemp  Dlnunemp  \
czone year                                                                
100.0 1990          0.043565 -0.074828     0.187104  0.124492 -0.050500   
      2000          0.009035 -0.285232     0.059469 -0.021407  0.482504   
      2008               NaN       NaN          NaN       NaN       NaN   
200.0 1990          0.001319  0.164829     0.318078  0.308975  0.114972   
      2000          0.021592 -0.378787     0.228724  0.101514  0.362970   

             Dlnnilf    Dlnpop  
czone year                      
100.0 1990  0.439338  0.170114  
      2000  0.070228  0.025113  
      2008       NaN       NaN  
200.0 1990  0.324466  0.303317  
      2000  0.436514  0.150242  

[5 rows x 27 columns]

#### Name variables to be consistent with the ADH replication file and merge the explanatory variables

In [33]:
for c in share_cols:
    df_cy['D{}'.format(c)] = df_cy['D{}'.format(c)] * 100.0
    df_cy['D{}_col'.format(c)] = df_cy['D{}_col'.format(c)] * 100.0
    df_cy['D{}_ncol'.format(c)] = df_cy['D{}_ncol'.format(c)] * 100.0

# Multiply by 100 b/c reports log points:
cols_mask = df_cy.columns.str.contains('Dln')
for c in df_cy.columns[cols_mask]:
    df_cy[c] = df_cy[c] * 100.0
    
ADHnames = {
    # outcome for Table 3
    'Dmanuf_share' : 'd_sh_empl_mfg',

    # outcomes for Table 5
    # panel A
    'Dlnmanuf' : 'lnchg_no_empl_mfg',
    'Dlnnonmanuf' : 'lnchg_no_empl_nmfg',
    'Dlnunemp' : 'lnchg_no_unempl',
    'Dlnnilf' : 'lnchg_no_nilf',
    # panel B
    'Dmanuf_share' : 'd_sh_empl_mfg',
    'Dnonmanuf_share' : 'd_sh_empl_nmfg',
    'Dunemp_share' : 'd_sh_unempl',
    'Dnilf_share' : 'd_sh_nilf',  
    # panel C
    'Dmanuf_share_col' : 'd_sh_empl_mfg_edu_c',
    'Dnonmanuf_share_col' : 'd_sh_empl_nmfg_edu_c',
    'Dunemp_share_col' : 'd_sh_unempl_edu_c',
    'Dnilf_share_col' : 'd_sh_nilf_edu_c',
    # panel D
    'Dmanuf_share_ncol' : 'd_sh_empl_mfg_edu_nc',
    'Dnonmanuf_share_ncol' : 'd_sh_empl_nmfg_edu_nc',
    'Dunemp_share_ncol' : 'd_sh_unempl_edu_nc',
    'Dnilf_share_ncol' : 'd_sh_nilf_edu_nc',
    
    # outcomes for Table 6
    'Dlnwkwage' : 'd_avg_lnwkwage',
    'Dlnwkwage_male' : 'd_avg_lnwkwage_m',
    'Dlnwkwage_female' : 'd_avg_lnwkwage_f',
    'Dlnwkwage_col' : 'd_avg_lnwkwage_c',
    'Dlnwkwage_ncol' : 'd_avg_lnwkwage_nc',
    'Dlnwkwage_col_male' : 'd_avg_lnwkwage_c_m',
    'Dlnwkwage_col_female' : 'd_avg_lnwkwage_c_f',
    'Dlnwkwage_ncol_male' : 'd_avg_lnwkwage_nc_m',
    'Dlnwkwage_ncol_female' : 'd_avg_lnwkwage_nc_f'
}

df_cy.rename(columns=ADHnames, inplace=True)

In [34]:
df_cy.head()

d_avg_lnwkwage  d_avg_lnwkwage_c  d_avg_lnwkwage_nc  \
czone year                                                        
100.0 1990       33.524195         35.338747          33.093125   
      2000       20.181813         28.755449          18.084823   
      2008             NaN               NaN                NaN   
200.0 1990       33.815071         29.811152          34.620654   
      2000       14.777333         18.513383          14.088373   

            d_avg_lnwkwage_m  d_avg_lnwkwage_f  d_avg_lnwkwage_c_m  \
czone year                                                           
100.0 1990         29.783407         38.739263           33.167230   
      2000         19.544325         21.081317           27.686632   
      2008               NaN               NaN                 NaN   
200.0 1990         27.626973         42.214231           25.240723   
      2000         13.876953         16.114992           21.288842   

            d_avg_lnwkwage_c_f  d_avg_lnwkwage_nc_m  d_avg_lnwkwage_nc_f  \
czone year                                                                 
100.0 1990           38.315482            29.005998            38.828287   
      2000           30.298831            17.612952            18.736626   
      2008                 NaN                  NaN                  NaN   
200.0 1990           35.628441            28.159564            43.482986   
      2000           15.317095            12.693518            16.084526   

            d_sh_empl_mfg  ...  d_sh_empl_mfg_edu_nc  d_sh_empl_nmfg_edu_nc  \
czone year                 ...                                                
100.0 1990      -4.472535  ...             -4.996556               1.740155   
      2000      -4.673979  ...             -5.554960               2.022482   
      2008            NaN  ...                   NaN                    NaN   
200.0 1990      -1.145592  ...             -0.775343               1.880722   
      2000      -8.409851  ...             -9.991662               6.458576   

            d_sh_unempl_edu_nc  d_sh_nilf_edu_nc  lnchg_no_empl_mfg  \
czone year                                                            
100.0 1990           -1.100090          4.356491          -7.482758   
      2000            2.628934          0.903544         -28.523237   
      2008                 NaN               NaN                NaN   
200.0 1990           -1.237259          0.131880          16.482932   
      2000            1.373846          2.159240         -37.878669   

            lnchg_no_empl_nmfg     Dlnemp  lnchg_no_unempl  lnchg_no_nilf  \
czone year                                                                  
100.0 1990           18.710384  12.449203        -5.049981      43.933770   
      2000            5.946904  -2.140713        48.250414       7.022836   
      2008                 NaN        NaN              NaN            NaN   
200.0 1990           31.807798  30.897548        11.497169      32.446556   
      2000           22.872398  10.151399        36.296955      43.651429   

               Dlnpop  
czone year             
100.0 1990  17.011376  
      2000   2.511251  
      2008        NaN  
200.0 1990  30.331677  
      2000  15.024204  

[5 rows x 27 columns]

In [36]:
# Original non-CA data:
df_NCA = pd.read_stata('workfile_china.dta')

# CA data:
CA_cols = [v for k,v in ADHnames.items()]
other_cols = df_NCA.columns.difference(CA_cols)
df_CA = pd.merge(df_cy, df_NCA[other_cols], 
                 left_on=['czone','year'], right_on=['czone','yr'], how='inner')

del df_cy

## Run Regressions!

In [37]:
def MyIVreg(formula, df):
    res = IV2SLS.from_formula(
        formula,
        df,
        weights = df['timepwt48']
    ).fit(cov_type="clustered", clusters=df["statefip"])
    
    return res

In [38]:
# pd.options.display.latex.repr = True

def CompareDF(x, fit_stats=['Estimator', 'R-squared', 'No. Observations'], keep=[]):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        y = pd.read_csv(StringIO(compare(x, stars=True, precision='std_errors').summary.as_csv()), 
                        skiprows=1, skipfooter=1, engine='python')
    z = pd.DataFrame(
        data=y.iloc[:, 1:].values,
        index=y.iloc[:, 0].str.strip(),
        columns=pd.MultiIndex.from_arrays(
            arrays=[y.columns[1:], y.iloc[0][1:]],
            names=['Model', 'Dep. Var.']
        )
    )
    if not keep:
        return pd.concat([z.iloc[11:], z.loc[fit_stats]])
    else:
        return pd.concat([*[z.iloc[z.index.get_loc(v):z.index.get_loc(v)+2] for v in keep], z.loc[fit_stats]])

In [39]:
def Table3(df):
    regions = list(filter(lambda x: x.startswith("reg"), df.columns))
    controls = [
        ["t2"],
        ["t2","l_shind_manuf_cbp"],
        ["t2","l_shind_manuf_cbp"] + regions,
        ["t2","l_shind_manuf_cbp", "l_sh_popedu_c", "l_sh_popfborn", "l_sh_empl_f"] + regions,
        ["t2","l_shind_manuf_cbp", "l_task_outsource", "l_sh_routine33"] + regions,
        ["t2","l_shind_manuf_cbp", "l_sh_popedu_c", "l_sh_popfborn", "l_sh_empl_f", "l_task_outsource", "l_sh_routine33"] + regions,
    ]

    baseform = "d_sh_empl_mfg ~ [d_tradeusch_pw ~ d_tradeotch_pw_lag] + 1"
    models = {
    '({})'.format(i+1) : ' + '.join([baseform, *controls[i]]) for i in range(len(controls))
    }
    res = {i : MyIVreg(m,df) for i, m in models.items()}
 
    baseform_first = 'd_tradeusch_pw ~ d_tradeotch_pw_lag + 1'
    models_first = {
        '({})'.format(i+1) : ' + '.join([baseform_first, *controls[i]]) 
        for i in range(len(controls))
    }
    res_first = {i : MyIVreg(m,df) for i, m in models_first.items()}

    return res, res_first

In [40]:
def Table5(df):
    regions = list(filter(lambda x: x.startswith("reg"), df.columns))
    controls = ['t2','l_shind_manuf_cbp','l_sh_popedu_c','l_sh_popfborn','l_sh_empl_f','l_sh_routine33',
                 'l_task_outsource'] + regions
    lhs = {
#         'A':['lnchg_no_empl_mfg','lnchg_no_empl_nmfg','lnchg_no_unempl','lnchg_no_nilf','lnchg_no_ssadiswkrs'],
#         'B':['d_sh_empl_mfg','d_sh_empl_nmfg','d_sh_unempl','d_sh_nilf','d_sh_ssadiswkrs'],
        'A':['lnchg_no_empl_mfg','lnchg_no_empl_nmfg','lnchg_no_unempl','lnchg_no_nilf'],
        'B':['d_sh_empl_mfg','d_sh_empl_nmfg','d_sh_unempl','d_sh_nilf'],        
        'C':['d_sh_empl_mfg_edu_c','d_sh_empl_nmfg_edu_c','d_sh_unempl_edu_c','d_sh_nilf_edu_c'],
        'D':['d_sh_empl_mfg_edu_nc','d_sh_empl_nmfg_edu_nc','d_sh_unempl_edu_nc','d_sh_nilf_edu_nc']
    }
    models_a = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['A'][i]), 
                                         *controls]) for i in range(len(lhs['A']))
    }
    models_b = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['B'][i]), 
                                         *controls]) for i in range(len(lhs['B']))
    }
    models_c = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['C'][i]), 
                                         *controls]) for i in range(len(lhs['C']))
    }
    models_d = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['D'][i]), 
                                         *controls]) for i in range(len(lhs['D']))
    }

    res_a = {i : MyIVreg(m,df) for i, m in models_a.items()}
    res_b = {i : MyIVreg(m,df) for i, m in models_b.items()}
    res_c = {i : MyIVreg(m,df) for i, m in models_c.items()}
    res_d = {i : MyIVreg(m,df) for i, m in models_d.items()}

    return res_a, res_b, res_c, res_d

In [41]:
def Table6(df):
    regions = list(filter(lambda x: x.startswith("reg"), df.columns))
    controls = ['t2','l_shind_manuf_cbp','l_sh_popedu_c','l_sh_popfborn','l_sh_empl_f','l_sh_routine33',
                 'l_task_outsource'] + regions
    lhs = {
        'A':['d_avg_lnwkwage','d_avg_lnwkwage_m','d_avg_lnwkwage_f'],
        'B':['d_avg_lnwkwage_c','d_avg_lnwkwage_c_m','d_avg_lnwkwage_c_f'],
        'C':['d_avg_lnwkwage_nc','d_avg_lnwkwage_nc_m','d_avg_lnwkwage_nc_f'],
    }   
    models_a = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['A'][i]), 
                                         *controls]) for i in range(len(lhs['A']))
    }
    models_b = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['B'][i]), 
                                         *controls]) for i in range(len(lhs['B']))
    }
    models_c = {
        '({})'.format(i+1) : ' + '.join(['{} ~ 1 + [d_tradeusch_pw ~ d_tradeotch_pw_lag]'.format(lhs['C'][i]), 
                                         *controls]) for i in range(len(lhs['C']))
    }
    res_a = {i : MyIVreg(m,df) for i, m in models_a.items()}
    res_b = {i : MyIVreg(m,df) for i, m in models_b.items()}
    res_c = {i : MyIVreg(m,df) for i, m in models_c.items()}
    
    return res_a, res_b, res_c

### Table 3: Change in Manuf/Pop, Pooled Regressions with Controls

#### I. 1990–2007 stacked first differences

In [42]:
keep = ['d_tradeusch_pw','l_shind_manuf_cbp', 'l_sh_popedu_c', 'l_sh_popfborn', 'l_sh_empl_f', 
        'l_task_outsource', 'l_sh_routine33']
CompareDF(Table3(df_CA)[0], keep = keep)

Model,(1),(2),(3),(4),(5),(6)
Dep. Var.,d_sh_empl_mfg,d_sh_empl_mfg,d_sh_empl_mfg,d_sh_empl_mfg,d_sh_empl_mfg,d_sh_empl_mfg
,,,,,,
d_tradeusch_pw,-0.7871***,-0.6580***,-0.5897***,-0.5501***,-0.6054***,-0.6397***
,(0.0848),(0.1177),(0.1175),(0.0998),(0.1168),(0.1198)
l_shind_manuf_cbp,,-0.0338,-0.0498***,-0.0710***,-0.0566***,-0.0484***
,,(0.0210),(0.0193),(0.0161),(0.0151),(0.0127)
l_sh_popedu_c,,,,-0.0333*,,-0.0079
,,,,(0.0184),,(0.0126)
l_sh_popfborn,,,,-0.0062,,0.0362***
,,,,(0.0085),,(0.0128)


**Interpretation**. In Column 1 we are estimating 
$$ 100 \times \Delta L^m_{it} = \alpha + \beta \Delta IPW_{uit} + \gamma_t + e_{it} $$
where $L^m_{it}$ is (manufacturing employment)/(working-age population) and  $IPW_{uit}$ is the import exposure per worker measured in 1,000s of dollars (see Appendix Table 1 of ADH). Then an estimate $\widehat{\beta}=-0.7871$ means that an exogenous increase of $1,000 in exposure per worker leads to a predicted decrease of 0.79 percentage points in manufacturing employment per working-age population.

In [43]:
# 2SLS by Frisch-Waugh-Lovell - Column 3 of Table 3
import statsmodels.api as sm

# Residualize on controls:
regions = list(filter(lambda x: x.startswith("reg"), df_CA.columns))
controls = ["t2","l_shind_manuf_cbp"] + regions
W = sm.add_constant(df_CA[controls])
r_x = sm.WLS(df_CA['d_tradeusch_pw'], W, weights = df_CA['timepwt48']).fit().resid
r_y = sm.WLS(df_CA['d_sh_empl_mfg'], W, weights = df_CA['timepwt48']).fit().resid
r_z = sm.WLS(df_CA['d_tradeotch_pw_lag'], W, weights = df_CA['timepwt48']).fit().resid

# Predict X with Z:
x_hat = sm.WLS(r_x, r_z, weights = df_CA['timepwt48']).fit().predict()

# Regress Y on predicted X:
sm.WLS(r_y, x_hat, weights = df_CA['timepwt48']).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 WLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.078
Model:                            WLS   Adj. R-squared (uncentered):              0.077
Method:                 Least Squares   F-statistic:                              121.5
Date:                Tue, 29 Oct 2024   Prob (F-statistic):                    3.51e-27
Time:                        22:53:49   Log-Likelihood:                         -3541.1
No. Observations:                1444   AIC:                                      7084.
Df Residuals:                    1443   BIC:                                      7090.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5897      0.053    -11.022      0.000      -0.695      -0.485
==============================================================================
Omnibus:                      529.829   Durbin-Watson:                   1.901
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7236.889
Skew:                          -1.318   Prob(JB):                         0.00
Kurtosis:                      13.646   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### II. 2SLS first stage estimates

In [44]:
CompareDF(Table3(df_CA)[1], keep=['d_tradeotch_pw_lag'], fit_stats=['R-squared'])

Model,(1),(2),(3),(4),(5),(6)
Dep. Var.,d_tradeusch_pw,d_tradeusch_pw,d_tradeusch_pw,d_tradeusch_pw,d_tradeusch_pw,d_tradeusch_pw
,,,,,,
d_tradeotch_pw_lag,0.7916***,0.6637***,0.6518***,0.6346***,0.6380***,0.6310***
,(0.0793),(0.0881),(0.0924),(0.0925),(0.0893),(0.0900)
R-squared,0.5436,0.5729,0.5789,0.5846,0.5833,0.5848


### Table 5: Change in Employment, Unemployment and Non-Employment

In [45]:
results5a, results5b, results5c, results5d = Table5(df_CA)

#### Panel A. 100 × log change in population counts

In [46]:
CompareDF(results5a, keep=['d_tradeusch_pw'], fit_stats=[])

Model,(1),(2),(3),(4)
Dep. Var.,lnchg_no_empl_mfg,lnchg_no_empl_nmfg,lnchg_no_unempl,lnchg_no_nilf
,,,,
d_tradeusch_pw,-4.8311***,-0.0127,5.8837***,3.2958***
,(1.2155),(0.6671),(1.1384),(1.1033)


#### Panel B. Change in population shares

In [47]:
CompareDF(results5b, keep=['d_tradeusch_pw'], fit_stats=[])

Model,(1),(2),(3),(4)
Dep. Var.,d_sh_empl_mfg,d_sh_empl_nmfg,d_sh_unempl,d_sh_nilf
,,,,
d_tradeusch_pw,-0.6397***,-0.1175,0.2183***,0.5389***
,(0.1198),(0.1174),(0.0528),(0.1210)


#### College education

In [48]:
CompareDF(results5c, keep=['d_tradeusch_pw'], fit_stats=[])

Model,(1),(2),(3),(4)
Dep. Var.,d_sh_empl_mfg_edu_c,d_sh_empl_nmfg_edu_c,d_sh_unempl_edu_c,d_sh_nilf_edu_c
,,,,
d_tradeusch_pw,-0.5097***,0.1653,0.0518**,0.2926***
,(0.1587),(0.1555),(0.0251),(0.0836)


#### No college education

In [49]:
CompareDF(results5d, keep=['d_tradeusch_pw'], fit_stats=[])

Model,(1),(2),(3),(4)
Dep. Var.,d_sh_empl_mfg_edu_nc,d_sh_empl_nmfg_edu_nc,d_sh_unempl_edu_nc,d_sh_nilf_edu_nc
,,,,
d_tradeusch_pw,-0.6407***,-0.2645*,0.2677***,0.6376***
,(0.1065),(0.1460),(0.0708),(0.1601)


### Table 6: Wage Changes

In [50]:
results6a, results6b, results6c = Table6(df_CA)

#### Panel A. All education levels

In [51]:
CompareDF(results6a, keep=['d_tradeusch_pw'], fit_stats=['R-squared'])

Model,(1),(2),(3)
Dep. Var.,d_avg_lnwkwage,d_avg_lnwkwage_m,d_avg_lnwkwage_f
,,,
d_tradeusch_pw,-1.2216***,-1.2035***,-1.2584***
,(0.2900),(0.3288),(0.2850)
R-squared,0.5633,0.4346,0.6840


#### Panel B. College education

In [52]:
CompareDF(results6b, keep=['d_tradeusch_pw'], fit_stats=['R-squared'])

Model,(1),(2),(3)
Dep. Var.,d_avg_lnwkwage_c,d_avg_lnwkwage_c_m,d_avg_lnwkwage_c_f
,,,
d_tradeusch_pw,-0.9031***,-1.1292***,-0.5981*
,(0.3338),(0.3722),(0.3488)
R-squared,0.3121,0.1755,0.4005


#### Panel C. No college education

In [53]:
CompareDF(results6c, keep=['d_tradeusch_pw'], fit_stats=['R-squared'])

Model,(1),(2),(3)
Dep. Var.,d_avg_lnwkwage_nc,d_avg_lnwkwage_nc_m,d_avg_lnwkwage_nc_f
,,,
d_tradeusch_pw,-1.1817***,-1.1040***,-1.3044***
,(0.2662),(0.3118),(0.2707)
R-squared,0.6027,0.4813,0.7026
